pip install SentenceTransformers (https://huggingface.co/sentence-transformers)
conda install umap-learn
conda install hdbscan
conda install bertopic
conda install nltk

pip install SentenceTransformers (https://huggingface.co/sentence-transformers)
conda install umap-learn
conda install hdbscan
conda install bertopic
conda install nltk

In [39]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic

In [3]:
def extraire_contenu_tei(chemin_fichier):
    """
    Extrait le contenu d'un fichier XML TEI et le stocke sous forme structurée.
    
    Args:
        chemin_fichier: Chemin vers le fichier XML
        
    Returns:
        Dictionnaire contenant les données structurées
    """
    # Définir l'espace de noms TEI
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
    
    # Parser le fichier XML
    tree = ET.parse(chemin_fichier)
    root = tree.getroot()
    
    # Dictionnaire pour stocker les données
    donnees = {
        'fichier': os.path.basename(chemin_fichier)
    }
    
    # Fonction auxiliaire pour extraire le texte en excluant les notes
    def extraire_texte_sans_notes(element):
        if element.tag.endswith('note'):
            return ""
        
        texte = element.text or ""
        for enfant in element:
            texte += extraire_texte_sans_notes(enfant)
            if enfant.tail:
                texte += enfant.tail
        return texte
    
    # 1. Métadonnées de la lettre
    try:
        # ID de la lettre
        corresp_desc = root.find('.//tei:correspDesc', ns)
        if corresp_desc is not None:
            donnees['id_lettre'] = corresp_desc.get('{http://www.w3.org/XML/1998/namespace}id')
        
        # Expéditeur
        expediteur = root.find('.//tei:correspAction[@type="write"]/tei:persName', ns)
        if expediteur is not None:
            prenom = expediteur.find('./tei:forename', ns)
            nom = expediteur.find('./tei:surname', ns)
            if prenom is not None and nom is not None:
                donnees['expediteur'] = f"{prenom.text} {nom.text}"
        
        # Date
        date = root.find('.//tei:correspAction[@type="write"]/tei:date', ns)
        if date is not None:
            donnees['date'] = date.text
            donnees['date_when'] = date.get('when')
        
        # Destinataire
        destinataire = root.find('.//tei:correspAction[@type="received"]/tei:persName', ns)
        if destinataire is not None:
            prenom = destinataire.find('./tei:forename', ns)
            nom = destinataire.find('./tei:surname', ns)
            if prenom is not None and nom is not None:
                donnees['destinataire'] = f"{prenom.text} {nom.text}"
        
        # Mots-clés/thèmes
        mots_cles = root.findall('.//tei:textClass/tei:keywords/tei:term', ns)
        if mots_cles:
            donnees['mots_cles'] = [mot.text for mot in mots_cles if mot.text]
    except Exception as e:
        print(f"Erreur lors de l'extraction des métadonnées: {e}")
    
    # 2. Contenu textuel de la lettre
    try:
        contenu = []
        
        # Extraire l'ouverture (dateline, salutation)
        openers = root.findall('.//tei:opener', ns)
        for opener in openers:
            # Dateline
            dateline = opener.find('./tei:dateline', ns)
            if dateline is not None:
                texte = extraire_texte_sans_notes(dateline).strip()
                if texte:
                    contenu.append(texte)
            
            # Salutation
            salute = opener.find('./tei:salute', ns)
            if salute is not None:
                texte = extraire_texte_sans_notes(salute).strip()
                if texte:
                    contenu.append(texte)
        
        # Extraire les paragraphes du corps
        for p in root.findall('.//tei:div[@type="letter"]/tei:p', ns):
            texte = extraire_texte_sans_notes(p).strip()
            if texte:
                contenu.append(texte)
        
        # Extraire la clôture (signature)
        closer = root.find('.//tei:closer', ns)
        if closer is not None:
            # Salutation finale
            salute = closer.find('./tei:salute', ns)
            if salute is not None:
                texte = extraire_texte_sans_notes(salute).strip()
                if texte:
                    contenu.append(texte)
            
            # Signature
            signed = closer.find('./tei:signed', ns)
            if signed is not None:
                texte = extraire_texte_sans_notes(signed).strip()
                if texte:
                    contenu.append(texte)
        
        # Extraire les post-scriptum
        for ps in root.findall('.//tei:postscript', ns):
            for p in ps.findall('./tei:p', ns):
                texte = extraire_texte_sans_notes(p).strip()
                if texte:
                    contenu.append("P.S.: " + texte)
        
        # Joindre les éléments textuels
        donnees['texte_lettre'] = "\n\n".join([t for t in contenu if t])
    except Exception as e:
        print(f"Erreur lors de l'extraction du texte: {e}")
    
    # 3. Extraire les notes séparément (optionnel)
    # try:
    #     notes = []
    #     for note in root.findall('.//tei:note', ns):
    #         num = note.get('n', '')
    #         resp = note.get('resp', '')
    #         if note.text:
    #             notes.append(f"Note {num} {resp}: {note.text.strip()}")
        
    #     donnees['notes'] = "\n".join(notes)
    # except Exception as e:
    #     print(f"Erreur lors de l'extraction des notes: {e}")
    
    return donnees

def extraire_corpus_tei(chemin_dossier, motif="*.xml"):
    """
    Extrait le contenu de tous les fichiers XML TEI d'un dossier.
    
    Args:
        chemin_dossier: Chemin vers le dossier contenant les fichiers XML
        motif: Motif de filtrage des fichiers (par défaut "*.xml")
        
    Returns:
        DataFrame pandas avec les données extraites
    """
    # Liste pour stocker les données
    donnees = []
    
    # Récupérer tous les fichiers XML du dossier
    chemins_fichiers = glob(os.path.join(chemin_dossier, motif))
    
    for chemin in chemins_fichiers:
        try:
            # Extraire les données du fichier
            donnees_fichier = extraire_contenu_tei(chemin)
            donnees.append(donnees_fichier)
            print(f"Traitement réussi: {os.path.basename(chemin)}")
        except Exception as e:
            print(f"Erreur lors du traitement de {os.path.basename(chemin)}: {e}")
    
    # Créer le dataframe
    df = pd.DataFrame(donnees)
    return df

In [4]:
# Exemple d'utilisation pour un seul fichier
def exemple_fichier_unique(chemin_fichier):
    donnees = extraire_contenu_tei(chemin_fichier)
    df = pd.DataFrame([donnees])
    return df

# Exemple d'utilisation pour un dossier
def exemple_dossier(chemin_dossier):
    df = extraire_corpus_tei(chemin_dossier)
    return df

In [7]:
fichier = "/Users/Patrice/Proust/Lettres/03987_XVIII_307-637df4d1df8ed-66713160c9a90.xml"
df = exemple_fichier_unique(fichier)

In [78]:
dossier = "/Users/Patrice/Proust/Lettres/"
print(f"Traitement en cours... {dossier}")
df=exemple_dossier(dossier)


Traitement en cours... /Users/Patrice/Proust/Lettres/
Traitement réussi: 03139_XV_75_tei-6397065a691f2-6671315f8682d.xml
Traitement réussi: 01949_IX_24-664f138d2c963-6671315cc4462.xml
Traitement réussi: 02270_XI_6-6672ced2b3e43.xml
Traitement réussi: 03097_XV_33-637ce63e56269-6671315f16b61.xml
Traitement réussi: 05359_XXI_507-637df428499ce-66713161030a5.xml
Traitement réussi: 01979_IX_54-66dae809eee86.xml
Traitement réussi: 03872_XVIII_191-639b41b8cf79f-667131607e5f1.xml
Traitement réussi: 02996_XIV_108-639c89a2755d8-6671315eca7e3.xml
Traitement réussi: 03804_XVIII_123-637cf4a82f341-667131604cfb2.xml
Traitement réussi: 03789_XVIII_108-639c76166f8a9-66713160468ef.xml
Traitement réussi: 03836_XVIII_155-637798670d5d3-667131605a8dd.xml
Traitement réussi: 90035_XI_17-66be25fb3f117.xml
Traitement réussi: 03363_XVI_137_tei-639753b35be57-6671315fb98fc.xml
Traitement réussi: 03189_XV_ 125-637cf46218365-6671315f8f08d.xml
Traitement réussi: 03123_XV_59_tei-6396feba0e65d-6671315f4449b.xml
Traiteme

In [79]:
df.head(10)

,fichier,id_lettre,expediteur,date,date_when,destinataire,mots_cles,texte_lettre
0,03139_XV_75_tei-6397065a691f2-6671315f8682d.xml,cp03139,Lionel Hauser,le 14 juin 1916,19160614,Marcel Proust,[Corpus_Guerre_1914-1918],"Paris, le 14 Juin 1916\n ...."
1,01949_IX_24-664f138d2c963-6671315cc4462.xml,cp01949,Marcel Proust,mardi [le 2 mars 1909],19090302,"Robert Montesquiou, de","[pastiche, lectures]","Mardi\n\nCher Monsieur,\n\nJe vous remercie mi..."
2,02270_XI_6-6672ced2b3e43.xml,cp02270,Marcel Proust,[un peu avant la mi-janvier\n ...,19120115,Reynaldo Hahn,"[genèse, méta-épistolaire, lansgage et codes]",Je viens de causer toute la nuit avec toi en t...
3,03097_XV_33-637ce63e56269-6671315f16b61.xml,cp03097,Marcel Proust,le samedi 13 mai\n 1916,19160513,Lionel Hauser,[guerre],Samedi 13 \n Mai\n ...
4,05359_XXI_507-637df428499ce-66713161030a5.xml,cp05359,Marcel Proust,None,None,Jean Ajalbert,[prix Goncourt],"44 rue Hamelin\n\nMon cher Confrère etMaître,\..."
5,01979_IX_54-66dae809eee86.xml,cp01979,Marcel Proust,[vers mai\n 1909],None,Max Daireaux,"[santé, mondanités, épistolarité, genèse, dépl...",Cher ami\n\nComme je suis \n ...
6,03872_XVIII_191-639b41b8cf79f-667131607e5f1.xml,cp03872,Jacques-Émile Blanche,le 24 juillet 1919,19190724,Marcel Proust,[prix Goncourt],24 \n \n ...
7,02996_XIV_108-639c89a2755d8-6671315eca7e3.xml,cp02996,Marcel Proust,[vers le 9 septembre 1915],19150909,Antoine Bibesco,[],Cher Antoine\n\nJe te dirai comme\n ...
8,03804_XVIII_123-637cf4a82f341-667131604cfb2.xml,cp03804,Marcel Proust,[le lundi 16 juin\n 1919],19190616,"Robert Flers, de",[],8bis rue Laurent Pichat\n\nMon cher Robert\n\n...
9,03789_XVIII_108-639c76166f8a9-66713160468ef.xml,cp03789,Marcel Proust,None,19190528,Jacques Porel,[prix Goncourt],Cher ami\n\nL’électricien envoyé par la\n ...


In [80]:
df.to_csv("corpus_proust.csv", index=False, sep=";")
df.to_excel("corpus_proust.xlsx", index=False)